# Praca domowa 4 - zadanie dodatkowe

### Wczytanie danych

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("allegro-api-transactions.csv")
df = df[['main_category', 'categories', 'it_location', 'price']]

In [3]:
X = df.drop('price', axis = 1)
y = df.price

In [4]:
X.nunique()

main_category       27
categories        9020
it_location      10056
dtype: int64

In [5]:
X.shape

(420020, 3)

In [6]:
X.head()

,main_category,categories,it_location
0,Komputery,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",Warszawa
1,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",Warszawa
2,Dom i Ogród,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",Leszno
3,Książki i Komiksy,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",Wola Krzysztoporska
4,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",BIAŁYSTOK


In [7]:
X['it_location'] = X['it_location'].apply(lambda x: x.lower())

In [8]:
X.nunique()

main_category      27
categories       9020
it_location      7903
dtype: int64

In [9]:
X.head()

,main_category,categories,it_location
0,Komputery,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",warszawa
1,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",warszawa
2,Dom i Ogród,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",leszno
3,Książki i Komiksy,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",wola krzysztoporska
4,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",białystok


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

### Target encoding

Jednym z parametrów target encodingu jest smoothing. 

Pomaga on zapobiec przeuczeniu, jest on użyteczny w następującym przypadku: mamy kategorię, która przykładowo pojawia się tylko dwukrotnie w całym zbiorze. Opieranie predykcji na średniej z tych dwóch wyników jest bardziej narażone na overfitting niż w przypadku, gdy dla danej kategorii mamy 1000 obserwacji. Dlatego używa się wspomnianego parametru smoothing: w wyliczanej średniej uwzględniana jest wtedy średnia całego zbioru. Wartość tego parametru to waga, jaką przypisuje się  ogólnej średniej.

#### LinearRegression

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import category_encoders as ce

pipe1 = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('model', LinearRegression())
])

In [12]:
pipe1.fit(X_train, y_train)
print("Finished!")

Finished!


In [13]:
from sklearn.metrics import mean_squared_error as rmse

rmse(y_test, pipe1.predict(X_test), squared = False)

540.1465349761147

In [14]:
print(f"Wspołczynniki modelu: {pipe1.steps[1][1].coef_}")

Wspołczynniki modelu: [0.01023362 1.18312479 0.40704812]


### Regularyzacja

Do regularyzacji zastosuję dwa modele liniowe: **Lasso** i **Ridge**. Obie metody dbają o to, by normy wartości współczynników były mniejsze: w ich funkcji straty normy wyznaczanonych współczynników są fragmentem sumy. W przypadku **Ridge** norma występuje w kwadracie, natomiast w przypadku **Lasso** jest to po prostu norma. Każda z tych wartości jest mnożona przez perparametr **alpha**, który jest hiperparametrem modelu.

#### Lasso

In [15]:
from sklearn.linear_model import Lasso

pipe2 = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('model', Lasso(alpha = 20))
])

In [16]:
pipe2.fit(X_train, y_train)
print("Finished!")

Finished!


In [17]:
rmse(y_test, pipe2.predict(X_test), squared = False)

540.1485431661094

In [18]:
print(f"Wspołczynniki modelu: {pipe2.steps[1][1].coef_}")

Wspołczynniki modelu: [0.         1.18319441 0.40078774]


#### Ridge

In [19]:
from sklearn.linear_model import Ridge

pipe3 = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('model', Ridge(alpha = 20))
])

In [20]:
pipe3.fit(X_train, y_train)
print("Finished!")

Finished!


In [21]:
rmse(y_test, pipe3.predict(X_test), squared = False)

540.1465349655437

In [22]:
print(f"Wspołczynniki modelu: {pipe3.steps[1][1].coef_}")

Wspołczynniki modelu: [0.01023362 1.18312479 0.40704811]


Zauważmy, że **Lasso** wyzerował pierwszą kolumnę. Wydaje się to logiczne, że ta kolumna ma najmniejsze znaczenie. Wskazuje ona główną kategorię, która musi już być w pewien sposób zakodowana przez kolumnę categories (kolumna categories, która zawiera więcej niż jedną kategorię, dokładniej wskazuje rodzaj produktu niż tylko jedna, główna kategoria).

Wszystkie trzy modele wymienione wyżej osiągnęły bardzo podobny błąd RMSE. Współczynniki **Ridge** oraz zwyczajnej **regresji liniowej** są również bardzo podobne (prawie identyczne). Wartości współczynników **Lasso** również są podobne, z tą różnicą (co wcześniej napisałem), że jedna kolumna została wyzerowana.

### Random Forest Regressor

In [23]:
from sklearn.ensemble import RandomForestRegressor

pipe4 = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('model', RandomForestRegressor(n_jobs = -1, random_state = 123))
])

In [24]:
pipe4.fit(X_train, y_train)
print("Finished!")

Finished!


In [25]:
rmse(y_test, pipe4.predict(X_test), squared = False)

516.1378548895078

### XGBoost Regressor

In [26]:
import xgboost as xgb

pipe5 = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('model', xgb.XGBRegressor(n_jobs = -1, random_state = 123))
])

In [27]:
pipe5.fit(X_train, y_train)
print("Finished!")

Finished!


In [28]:
rmse(y_test, pipe5.predict(X_test), squared = False)

504.0022544433332

Zanim przejdę do wyboru najlepszego modelu, sprawdzę jeszcze jak wszystkie cztery modele radzą sobie bez pierwszej kolumny (którą **Lasso** wyzerowało).

In [29]:
from sklearn.compose import ColumnTransformer

selector = Pipeline([
    ("selector", ColumnTransformer([("selector", "passthrough", ['categories', 'it_location'])], remainder = "drop"))
])

#### Linear Regression

In [30]:
pipe1a = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('selector', selector),
    ('model', LinearRegression())
])

pipe1a.fit(X_train, y_train)

print(f"RMSE: {rmse(y_test, pipe1a.predict(X_test), squared = False)}")
print(f"Wspołczynniki modelu: {pipe1a.steps[2][1].coef_}")

RMSE: 540.1503657822451
Wspołczynniki modelu: [1.18359003 0.40730326]


#### Lasso

In [31]:
pipe2a = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('selector', selector),
    ('model', Lasso(alpha = 20))
])

pipe2a.fit(X_train, y_train)

print(f"RMSE: {rmse(y_test, pipe2a.predict(X_test), squared = False)}")
print(f"Wspołczynniki modelu: {pipe2a.steps[2][1].coef_}")

RMSE: 540.1485374359431
Wspołczynniki modelu: [1.18319133 0.40078997]


#### Ridge

In [32]:
pipe3a = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('selector', selector),
    ('model', Ridge(alpha = 20))
])

pipe3a.fit(X_train, y_train)

print(f"RMSE: {rmse(y_test, pipe3a.predict(X_test), squared = False)}")
print(f"Wspołczynniki modelu: {pipe3a.steps[2][1].coef_}")

RMSE: 540.150365773021
Wspołczynniki modelu: [1.18359003 0.40730326]


#### Random Forest Regressor

In [33]:
pipe4a = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('selector', selector),
    ('model', RandomForestRegressor(n_jobs = -1, random_state = 123))
])

pipe4a.fit(X_train, y_train)

print(f"RMSE: {rmse(y_test, pipe4a.predict(X_test), squared = False)}")

RMSE: 535.1622701904357


#### XGBoost Regressor

In [34]:
pipe5a = Pipeline([
    ('target_encoding', ce.TargetEncoder(smoothing = 20)),
    ('selector', selector),
    ('model', xgb.XGBRegressor(n_jobs = -1, random_state = 123))
])

pipe5a.fit(X_train, y_train)

print(f"RMSE: {rmse(y_test, pipe5a.predict(X_test), squared = False)}")

RMSE: 528.0085955968258


### Wybranie najlepszego modelu

W tym celu skorzystam z dwóch miar: RMSE oraz R2.

In [35]:
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as rmse

In [36]:
def append_result(df, name, clf, X_test, y_test):
    df = df.append({
        'model' : name,
        'rmse' : rmse(y_test, clf.predict(X_test), squared = False),
        'r2' : r2(y_test, clf.predict(X_test))
    }, ignore_index = True)
    return df

In [37]:
results = pd.DataFrame(columns = ['model', 'rmse', 'r2'])

In [38]:
results = append_result(results, 'LinearRegression', pipe1, X_test, y_test)
results = append_result(results, 'Lasso', pipe2, X_test, y_test)
results = append_result(results, 'Ridge', pipe3, X_test, y_test)
results = append_result(results, 'RandomForestRegressor', pipe4, X_test, y_test)
results = append_result(results, 'XGBoostRegressor', pipe5, X_test, y_test)

results = append_result(results, 'LinearRegression_two_columns', pipe1a, X_test, y_test)
results = append_result(results, 'Lasso_two_columns', pipe2a, X_test, y_test)
results = append_result(results, 'Ridge_two_columns', pipe3a, X_test, y_test)
results = append_result(results, 'RandomForestRegressor_two_columns', pipe4a, X_test, y_test)
results = append_result(results, 'XGBoostRegressor_two_columns', pipe5a, X_test, y_test)

In [40]:
results.sort_values(by = 'rmse')

,model,rmse,r2
4,XGBoostRegressor,504.002254,0.184753
3,RandomForestRegressor,516.137855,0.145021
9,XGBoostRegressor_two_columns,528.008596,0.105241
8,RandomForestRegressor_two_columns,535.162270,0.080831
2,Ridge,540.146535,0.063630
0,LinearRegression,540.146535,0.063630
6,Lasso_two_columns,540.148537,0.063623
1,Lasso,540.148543,0.063623
7,Ridge_two_columns,540.150366,0.063617
5,LinearRegression_two_columns,540.150366,0.063617


Spośród wytrenowanych modeli najlepszym okazał się **XGBoostRegressor**, który korzysta ze wszystkich trzech kolumn Jego błąd RMSE jest najmniejszy, a wartość R2 największa spośród tych modeli. 

Należy zaznaczyć, że hiperparametry modeli nie były tunningowane. Możliwe, że po tunningu modele osiagnęłyby lepsze wyniki.